<img src="https://chengmo-dev1.bj.bcebos.com/page1.png" alt="drawing" width="1000"/>

# 前言 - 学习本项目你可以获得什么
- 理论学习：了解AIAgent的基础知识
- 上手实操：深入了解Agent中的FunctionCall运行流程
- 上手实操：入门百度智能云千帆AppBuilder，在十分钟内打造一个个性化AIAgent
- 上手实操：使用AppBuilder-SDK打造一个端云组件联动的进阶Agent

## 1. 项目背景

### 1.1、 什么是AppBuilder
[百度智能云千帆AppBuilder](https://appbuilder.cloud.baidu.com/)（以下简称AppBuilder）是基于大模型搭建AI原生应用的工作台，旨在降低AI原生应用的开发门槛，赋能开发者和企业快速实现应用搭建。

平台提供了RAG（检索增强生成）、Agent（智能体）等应用框架，内置了文档问答、表格问答、多轮对话、生成创作等多种应用组件，还包括百度搜索和百度地图等特色组件，以及文本处理、图像处理和语音处理等传统AI组件，支持零代码、低代码、全代码三种开发方式，满足不同开发能力的开发者和企业的场景需求。

### 1.2、 什么是AppBuilder-SDK

[百度智能云千帆AppBuilder-SDK](https://github.com/baidubce/app-builder)(以下简称AB-SDK)，百度智能云千帆AppBuilder-SDK是百度智能云千帆AppBuilder面向AI原生应用开发者提供的一站式开发平台的客户端SDK。

<img src="https://chengmo-dev1.bj.bcebos.com/page2.png" alt="drawing" width="1000"/>

# 2. 项目介绍 - 通过ToolCall实现端云组件联动的Agent
### 2.1、 什么是Agent

AIAgent是能够感知环境，基于目标进行决策并执行动作的智能化应用。不同于传统人工智能应用（主要指以规则引擎、机器学习、深度学习等技术为核心）和RPA机器人，AIAgent能够基于目标和对现状能力的认知，在环境约束中，依赖特定资源和现有工具，找到行动规则并将行动拆解为必要的步骤，自主执行步骤，达成目标。

AIAgent具备三个核心能力：独立思考、自主执行、持续迭代。
- 独立思考是指AlAgent能够根据给定任务目标和约束条件，进行任务规划和问题拆解，形成执行步骤（即工作流）；
- 自主执行是指AlAgent能够调取各类组件和工具，按照执行步骤依次执行，实现任务目标；
- 持续选代是指AlAgent能够自动记录任务目标、工作流和执行结果，基于结果反馈，沉淀专家知识和案例。

AICopilot、AIAgent、大模型等名词在各类文章上经常混淆，此处简要说明下三者的区别。大模型一般是指大模型技术，AlAgent和Al Copilot是基于大模型技术的智能化应用，AlAgent和AlCopilot在功能和场景上存在差别。

自主性是AIAgent和AI Copilot之间最大的区别。AI Copilot是“副驾驶”，只是提供建议而非决策，AIAgent是“主驾驶”，需要真正做出决策并开展行动。

<img src="https://chengmo-dev1.bj.bcebos.com/page3.png" alt="drawing" width="1000"/>

### 2.2、 什么是ToolCall

解释该问题，需要了解以下的知识点：`Agent工具` -> `FunctionCall` - `ToolCall`

AIAgent 有四大核心组件：记忆、规划、工具和执行。其中工具部分，与我们的开发关系最密切，在各类Agent开发平台/工具中，常被称为“组件”、"插件"、"能力"等.

关于Agent的工具的定义与分类，如下图~

<img src="https://chengmo-dev1.bj.bcebos.com/page4.png" alt="drawing" width="1000"/>

Agent使用工具的流程，一般称为`FunctionCall`，最早由OpenAI提出，并在[Assistant API](https://platform.openai.com/docs/assistants/overview)中广泛应用。


ToolCall，则是AppBuilder平台提出的一种进阶的FunctionCall，本质与OpenAI的FunctionCall一致，但具有以下两个特点：

- **端云组件联动**： Agent 调用工具时，可以同时调用云端和本地组件。

- **组件类型泛化**： AppBuilder在未来会支持多种类型组件，已经超出了Function的含义，例如数据库、记忆库、工作流等等



### 2.3、 什么是端云组件联动，要解决什么问题

我们首先从工具的执行位置出发展开~ 在使用如AppBuilder / Coze 等平台开发Agent时，我们可以使用很多平台组件广场中，官方提供的组件，这里组件开箱即用，非常方便。

<img src="https://chengmo-dev1.bj.bcebos.com/page5.png" alt="drawing" width="1000"/>

但是存在一个问题，基于平台云端组件开发的应用，无法调用内网/局域网/私域的知识与能力，也无法与本地的工具进行联动，限制了Agent的灵活性。

我们在解决实际业务问题时，常遇到需要访问内网链接API或本地/硬件功能的FunctionCall需求，AppBuilder ToolCall可以解决这个问题：

* 1、用户可注册一个本地运行的组件到已发布的应用
* 2、由AppBuilder-Agent的云端思考模型进行规划和参数生成
* 3、用户基于生成的参数调用本地组件，并再上传运行结果
* 4、以此实现将本地组件能力嵌入到应用整体流程


<img src="https://chengmo-dev1.bj.bcebos.com/page6.png" alt="drawing" width="1000"/>

# 3、ToolCall（FunctionCall）基础知识介绍

### 3.1、Agent是如何调用Tool的

我们可以将Agent的黑箱拆解为以下几个部分：
1. Agent的背景信息
2. Agent的输入信息
3. Agent的思考过程
4. Agent触发组件调用
5. Agent基于组件输出反思总结

#### Agent的背景信息包含以下几个部分
- 角色定义描述（Prompt）：定义Agent的角色
- 能力描述（Prompt）：定义Agent可以干什么
- 工具描述（JsonSchema/Str）：将工具的输入和输出，按照规范，定义为一段字符串，作为最终大模型Prompt的一部分
- **函数定义描述（functions）**：允许直接传入本地定义的函数对象列表，替代工具描述中的`JsonSchema`，在某些场景下可以简化调用流程，避免繁琐的JSON格式定义。

##### functions字段的引入与兼容性说明
为了提供更高的灵活性，Agent现在支持`functions`字段，允许传入本地函数对象的列表。相比于`tools`字段中的`JsonSchema`格式定义，`functions`字段可以更加直观地直接传递函数对象，适合无需复杂参数定义的场景。

**兼容性说明**：当`tools`和`functions`字段同时启用时，`tools`字段优先被调用。`functions`字段将在`tools`未定义时被调用，以保持兼容旧有代码的功能。
##### 使用functions字段的注意事项

当使用`functions`字段时，函数的注释（docstring）和参数的类型注解将自动被转换为JSON格式，以供Agent调用。因此，在定义函数时，请确保以下几点：

1. **注释（docstring）**：函数的文档字符串非常重要，它将作为大模型调用时的描述。如果缺少注释，函数的描述可能不完整或不准确，影响调用结果。
2. **参数类型注解**：所有函数参数都应提供明确的类型注解。中间件会根据这些类型注解将参数转换为JSON Schema中对应的类型（例如，`str`对应`string`，`int`对应`integer`等）。如果没有注解，默认会将参数类型设置为`string`。
3. **函数签名**：函数的签名（参数名称和类型）会被自动解析为JSON中的参数定义，确保函数参数使用合理的名称并提供类型提示。

例如，以下是一个符合要求的函数定义：

```python
def get_current_weather(location: str, unit: str) -> str:
    """
    查询指定中国城市的当前天气。

    参数:
        location (str): 城市名称，例如："北京"
        unit (str): 温度单位，可选 "celsius" 或 "fahrenheit"

    返回:
        str: 天气情况描述
    """
    return "北京今天25度"
```

#### Agent的输入信息包含以下几个部分
- 用户输入（Query/Prompt）：用户输入的文本
- 对话相关的文件（File/Url）：与本地对话相关的文件路径

#### Agent的思考过程
AppBuilder-Agent会将背景信息与输入信息，拼接为最终的Prompt，然后调用大模型推理。

Prompt的一个简单且直观的例子是：

你是`{角色定义描述}`，你可以做以下事情：`{能力描述}`，你可以使用这些工具：`{工具描述-description}`，工具依赖的输入是：`{工具描述-paramters-properties-name}`，这些输入的格式分别是`{工具描述-paramters-properties-type}`。现在用户的问题是`{用户输入}`，与该问题相关的文件是`{对话相关的文件}`，请你解决用户的这个问题。

#### Agent触发组件调用

如果用户的query和组件能够解决的问题匹配，那么大模型就会尝试根据prompt里给出的工具的描述，从query中提炼出该次调用工具所需的参数，生成一个ToolCall命令，交给执行组件的模块去执行。

例如，我们给出的组件能力是"查找公司内指定人员的信息"，函数的参数名为"name"。当用户输入"查找张三的信息"，大模型会从query中提炼出参数"name=张三"这个信息。

<img src="https://chengmo-dev1.bj.bcebos.com/page7.png" alt="drawing" width="1000"/>

#### Agent基于组件输出反思总结

组件运行模块执行组件后，会给出字符串形式的结果给到Agent，Agent会再次将结果拼接为Prompt，然后调用大模型推理。判断用户的需求是否已经解决。如果解决了，则经过一个对话模块，总结用户的需求，并生成一个对话记录。如果未解决，则继续调用大模型推理，尝试调用更多的工具，直到用户的需求被解决。

### 3.2、开发者如何命令Agent调用本地Tool

在AppBuilder-SDK中，开发者可以通过`ToolCall`功能注册Agent调用本地定义的工具或函数。为了提高灵活性，`ToolCall`现在支持两种方式来定义和调用工具：

1. **使用 `tools` 字段**：通过`JsonSchema`格式定义工具的输入和输出，适用于较为复杂的工具描述和调用。
2. **使用 `functions` 字段**：直接传递本地定义的函数对象，适用于简单的函数调用，避免繁琐的`JsonSchema`定义。

#### tools 和 functions 字段的优先级

当`tools`和`functions`字段同时定义时，系统将优先调用`tools`字段中的工具描述。如果`tools`未定义，则使用`functions`中的本地函数。

#### 示例：如何通过ToolCall命令Agent调用本地工具

以下代码示例分别展示了如何使用tools字段和functions字段来调用本地定义的函数。tools字段适用于通过JsonSchema定义复杂工具，而functions字段则简化了调用流程，直接传递本地函数对象。


```python
import appbuilder

# 实例化AppBuilderClient
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

# 第一次对话，输入原始的query 和 工具描述
message_1 = app_client.run(
    conversation_id=conversation_id,
    query="请问张三同学的生日是哪天？",
    tools=tools
)
tool_call = message_1.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id

# 第二次对话，在本地执行组件后，上传组件的运行结果
tool_call_result = "张三同学的生日是2008年8月8日"
message_2 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": tool_call_result
    }]
)
print(message_2.content)
```

其中`AppBuilderClient`的`run`方法是核心，我们展开该函数的定义和参数介绍：

`AppBuilderClient().run() -> Message`

```python
def run(self, conversation_id: str,
        query: str = "",
        file_ids: list = [],
        stream: bool = False,
        tools: list[data_class.Tool] = None,
        functions: list[Callable] = None,  # 新增 functions 字段
        tool_outputs: list[data_class.ToolOutput] = None,
        **kwargs
        ) -> Message:
    r"""
        参数:
            query (str: 必须): query内容
            conversation_id (str, 必须): 唯一会话ID，如需开始新的会话，请使用self.create_conversation创建新的会话
            file_ids(list[str], 可选):
            stream (bool, 可选): 为True时，流式返回，需要将message.content.answer拼接起来才是完整的回答；为False时，对应非流式返回
            tools(list[data_class.Tools], 可选): 一个Tools组成的列表，其中每个Tools对应一个工具的配置, 默认为None
            functions(list[Callable], 可选): 传递本地定义的函数对象列表，简化函数调用，默认为None  # 新增字段
            tool_outputs(list[data_class.ToolOutput], 可选): 工具输出列表，格式为list[ToolOutput], ToolOutput内容为本地的工具执行结果，以自然语言/json dump str描述，默认为None
        返回: message (obj: `Message`): 对话结果.
    """
    pass
```


| 参数名称        | 参数类型         | 是否必须 | 描述                                                         | 示例值            |
| --------------- | ---------------- | -------- | ------------------------------------------------------------ | ----------------- |
| conversation_id | String           | 是       | 会话ID                                                       |                   |
| query           | String           | 否       | query问题内容                                                | "今天天气怎么样?" |
| file_ids        | list[String]     | 否       | 对话可引用的文档ID                                           |                   |
| stream          | Bool             | 否       | 为true时则流式返回，为false时则一次性返回所有内容, 推荐设为true，降低首token时延 | False             |
| tools           | List[Tool]       | 否       | 一个列表，其中每个字典对应一个工具的配置                     |                   |
| functions       | List[Callable]   | 否       | 传入本地函数对象列表，以直接调用函数                         |                   |
| tools[0]        | Tool             | 否       | 工具配置                                                     |                   |
| +type           | String           | 否       | 枚举：<br/>**file_retrieval**: 知识库检索工具能够理解文档内容，支持用户针对文档内容的问答。<br/>**code_interpreter**: 代码解释器, 代码解释器能够生成并执行代码，从而协助用户解决复杂问题，涵盖科学计算（包括普通数学计算题）、数据可视化、文件编辑处理（图片、PDF文档、视频、音频等）、文件格式转换（如WAV、MP3、text、SRT、PNG、jpg、MP4、GIF、MP3等）、数据分析&清洗&处理（文件以excel、csv格式为主）、机器学习&深度学习建模&自然语言处理等多个领域。<br/>**function**: 支持fucntion call模式调用工具 |                   |
| +function       | Function         | 否       | Function工具描述<br/>仅当**type为**`**function**` 时需要且必须填写 |                   |
| ++name          | String           | 否       | 函数名<br/>只允许数字、大小写字母和中划线和下划线，最大长度为64个字符。一次运行中唯一。 |                   |
| ++description   | String           | 否       | 工具描述                                                     |                   |
| ++parameters    | Dict             | 否       | 工具参数, json_schema格式                                    |                   |
| tool_outputs    | List[ToolOutput] | 否       | 内容为本地的工具执行结果，以自然语言/json dump str描述       |                   |
| tool_outputs[0] | ToolOutput       | 否       | 工具执行结果                                                 |                   |
| +tool_call_id   | String           | 否       | 工具调用ID                                                   |                   |
| +output         | String           | 否       | 工具输出                                                     |                   |

`Tool`与`Function`是本地组件的描述，类型为object，其定义如下：

```python
class Tool(BaseModel):
    type: str = "function"
    function: Function = Field(..., description="工具信息")

class Function(BaseModel):
    name: str = Field(..., description="工具名称")
    description: str = Field(..., description="工具描述")
    parameters: dict = Field(..., description="工具参数, json_schema格式")
```

`ToolOutput`是本地组件的执行结果，需要再次上传到Agent，参与思考，类型为object，其定义如下：
```python
class ToolOutput(BaseModel):
    tool_call_id: str = Field(..., description="工具调用ID")
    output: str = Field(..., description="工具输出")

```

# 4、ToolCall的第一个例子

我们继续以上文中提到的查找张三生日为例，看一下完整的流程是怎么样的

### 前置工作，在AppBuilder平台上创建一个白板应用（可以跳过）

网页链接：https://appbuilder.cloud.baidu.com/

注册后，进入控制台：https://console.bce.baidu.com/ai_apaas/dialogHome

点击左上角的【创建应用】-> 【AI自动配置】，我们输入以下Prompt，自动生成Agent：`你是智能问题解决者，自动集成多种工具组件，解决用户各类问题`

<img src="https://chengmo-dev1.bj.bcebos.com/page8.png" alt="drawing" width="1000"/>

最终生成的Agent长这个样子：

<img src="https://chengmo-dev1.bj.bcebos.com/page9.png" alt="drawing" width="1000"/>

而后点击【发布】，分别在控制台的左侧【个人空间】获取`app_id`，在【我的密钥】获取`APPBUILDER_TOEN`后，就可以开始后续的操作了。

当然，下面的示例代码中，我们已经提供了可以直接运行的试用Token与App，你可以直接上手运行

In [12]:
import os
import appbuilder

# AppBuilder Token，此处为试用Token，速度Quota有限制，正式使用替换为您个人的Token
os.environ["APPBUILDER_TOKEN"] = "bce-v3/ALTAK-n5AYUIUJMarF7F7iFXVeK/1bf65eed7c8c7efef9b11388524fa1087f90ea58"

# 应用为：智能问题解决者
app_id = "b9473e78-754b-463a-916b-f0a9097a8e5f"
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

首次提问一个问题，应用不具备该能力，通过回答可以印证

- 由于并没有关于张三同学的信息，所以Agent无法实现查询

In [13]:
message_1 = app_client.run(
    conversation_id=conversation_id,
    query="请问本公司的张三同学的生日是哪天？",
)
print("Agent第一次回答: {}".format(message_1.content.answer))

Agent第一次回答: 很抱歉，我无法直接获取您提到的本公司张三同学的生日信息。建议您通过公司内部系统或联系相关部门负责人来获取准确信息。如果您有其他问题或需要帮助，请随时告诉我。


**output**
```
Agent第一次回答: 为了回答这个问题，我们首先需要明确几个关键点：

1. **问题理解**：
   - 需要确定的是“张三同学的生日”。

2. **工具选择**：
   - 由于问题涉及的是特定个人的信息（张三的生日），这通常不是通过工具或系统查询能得到的，而是需要通过公司内部的人事记录或直接询问张三本人来获取。

3. **解决方案生成**：
   - **步骤一**：首先，尝试访问公司的人事系统或员工档案，看是否有张三的生日信息记录。
   - **步骤二**：如果人事系统或员工档案中没有相关信息，或者你不具备访问权限，那么可以考虑直接询问张三本人或其同事，看是否有人知道他的生日。
   - **步骤三**：如果以上方法都不可行，还可以尝试联系公司的人力资源部门，看他们是否能提供相关信息。

4. **注意事项**：
   - 在尝试获取张三的生日信息时，要确保遵守公司的隐私政策和相关法律法规，不要侵犯张三的隐私权。
   - 如果张三不愿意透露他的生日信息，应尊重他的选择，并停止进一步询问。

5. **可能遇到的问题**：
   - 人事系统或员工档案中可能没有张三的生日信息。
   - 张三或其同事可能不愿意透露生日信息。
   - 人力资源部门可能因隐私政策而无法提供相关信息。

综上所述，要确定张三的生日，最直接且尊重隐私的方法是直接询问张三本人，或者通过公司正式渠道（如人力资源部门）在遵守隐私政策的前提下进行查询。
```

#### 1. 使用 `tools` 字段进行 function call 的示例：
##### 赋予应用一个本地查询组件能力

- 这里我们使用`info_dict`模拟一个数据库查询的返回结果。

In [9]:
def get_person_infomation(name: str):
    info_dict = {
        "张三": "1980年1月1日",
        "李四": "1975年12月31日",
        "刘伟": "1990年12月30日"
    }

    if name in info_dict:
        return f"您要查找的{name}的生日是：{info_dict[name]}"
    else:
        return f"您要查找的{name}的信息我们暂未收录，请联系管理员添加。"
    
# 创建工具的描述：json_schema格式
tools = [
    {
        "type": "function",
"function": {
    "name": "get_person_infomation",
    "description": "查找公司内指定人员的信息",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "人员名称，例如：张三、李四",
            },
        },
        "required": ["name"],
    },
},
    }
]
#定义函数列表
functions = [get_person_infomation]
function_map = {f.__name__: f for f in functions}

- 现在我们已经完成了本地tool组件的设计，接下来我们将tool的功能赋予Client应用

In [10]:
message_2 = app_client.run(
    conversation_id=conversation_id,
    query="请问本公司的张三同学的生日是哪天？",
    tools=tools
)
print("Agent的中间思考过程：")
print(message_2.content.events[-1].model_dump_json(indent=4))
print("Agent思考结束，等待我们上传本地结果\n")

Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "",
                "name": "get_person_infomation",
                "arguments": {
                    "name": "张三"
                },
                "usage": {
                    "prompt_tokens": 641,
                    "completion_tokens": 39,
                    "total_tokens": 680,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "07c403f3-502d-41ec-9815-a4498452feb1"
            },
            "used_tool": []
        }
    },
    "usage": null,
    "tool_calls": [
        {
            "id": "07c403f3-502d-41ec-9815-a4498452feb1",
            "type": "function",
            "function": {
                "name": "get_person_infomation",
                "arguments":

**output**
这部分输出为Client应用的思考过程
```
Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "用户想要查询公司内张三同学的生日信息，这个需求很明确，且背景信息也足够。我可以使用get_person_infomation工具来查找张三的生日信息。",
                "name": "get_person_infomation",
                "arguments": {
                    "name": "张三"
                },
                "usage": {
                    "prompt_tokens": 697,
                    "completion_tokens": 87,
                    "total_tokens": 784,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "c23309f7-e24a-4476-85e2-3ef9cfd4f6ed"
            },
            "used_tool": []
...
    ]
}
Agent思考结束，等待我们上传本地结果
```

- 大模型下发了调用本地函数的参数，我们使用这个参数调用本地函数

In [11]:
tool_call = message_2.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id
tool_call_name = tool_call.function.name
tool_call_argument = tool_call.function.arguments
local_func_result = function_map[name](**args)
print("local_func_result: {}\n".format(local_func_result))

local_func_result: 您要查找的张三的生日是：1980年1月1日



**output**
```
local_func_result: 您要查找的张三的生日是：1980年1月1日
```

- 向应用返回本地运行的结果，完成本地函数toolcall调用

In [ ]:
message_3 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": local_func_result
    }]
)
print("Agent 拥有了本地函数调用能力后，回答是: {}".format(message_3.content.answer))

**output**
```
Agent的中间思考过程：
{
    "code": 0,
    "message": "",
    "status": "interrupt",
    "event_type": "Interrupt",
    "content_type": "contexts",
    "detail": {
        "text": {
            "function_call": {
                "thought": "用户想要查询公司内张三同学的生日信息，这个需求很明确，且背景信息也足够。我可以使用get_person_infomation工具来查找张三的生日信息。",
                "name": "get_person_infomation",
                "arguments": {
                    "name": "张三"
                },
                "usage": {
                    "prompt_tokens": 697,
                    "completion_tokens": 87,
                    "total_tokens": 784,
                    "name": "ERNIE-4.0-Turbo-8K",
                    "type": "plan"
                },
                "tool_call_id": "c23309f7-e24a-4476-85e2-3ef9cfd4f6ed"
            },
            "used_tool": []
...
    ]
}
Agent思考结束，等待我们上传本地结果

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

local_func_result: 您要查找的张三的生日是：1980年1月1日

Agent 拥有了本地函数调用能力后，回答是: # 解决方案

## 问题分析

用户想要查询公司内张三同学的生日信息。这是一个明确且具体的需求，我们可以通过`get_person_infomation`工具来获取这一信息。

## 工具运用

1. **工具选择**：`get_person_infomation`
2. **参数设置**：

	* `name`：张三

3. **执行结果**：张三的生日是1980年1月1日。

## 解决方案步骤

1. 使用`get_person_infomation`工具，并设置参数`name`为“张三”。
2. 等待工具执行，并获取张三的生日信息。
3. 将获取到的生日信息（1980年1月1日）告知用户。

## 注意事项

* 确保在使用`get_person_infomation`工具时，输入的姓名与公司内部记录的姓名完全一致，以避免查询错误。
* 如果工具返回“未找到”或类似结果，请检查姓名是否有误或联系公司人事部门确认信息。

通过上述步骤，我们可以准确地回答用户的问题，并提供张三的生日信息。
```

#### 2. 使用 `functions` 字段进行 function call 的示例：

In [14]:
#定义函数
def get_person_infomation(name: str):
    info_dict = {
        "张三": "1980年1月1日",
        "李四": "1975年12月31日",
        "刘伟": "1990年12月30日"
    }

    if name in info_dict:
        return f"您要查找的{name}的生日是：{info_dict[name]}"
    else:
        return f"您要查找的{name}的信息我们暂未收录，请联系管理员添加。"
#定义函数列表
functions = [get_person_infomation]
function_map = {f.__name__: f for f in functions}

In [ ]:
message_2 = app_client.run(
    conversation_id=conversation_id,
    query="请问本公司的张三同学的生日是哪天？",
    functions=functions
    #tools=tools
)
print("Agent的中间思考过程：")
print(message_2.content.events[-1].model_dump_json(indent=4))
print("Agent思考结束，等待我们上传本地结果\n")

In [ ]:
tool_call = message_2.content.events[-1].tool_calls[-1]
tool_call_id = tool_call.id
tool_call_name = tool_call.function.name
tool_call_argument = tool_call.function.arguments
local_func_result = function_map[name](**args)
print("local_func_result: {}\n".format(local_func_result))

In [ ]:
message_3 = app_client.run(
    conversation_id=conversation_id,
    tool_outputs=[{
        "tool_call_id": tool_call_id,
        "output": local_func_result
    }]
)
print("Agent 拥有了本地函数调用能力后，回答是: {}".format(message_3.content.answer))

# 5、ToolCal第二个例子-调用本地工具并且代码更简洁

我们可以使用AppBuilderClient应用来执行tool_call操作，完成指定的命令，但是需要自己配置client的思考与运行流程，较为繁琐。SDK提供了使用AppBuilderEventHandler简化tool_call操作的功能

##### 配置运行环境&导入Client应用

In [1]:
import os
import appbuilder


# AppBuilder Token，此处为试用Token
os.environ["APPBUILDER_TOKEN"] = "bce-v3/ALTAK-n5AYUIUJMarF7F7iFXVeK/1bf65eed7c8c7efef9b11388524fa1087f90ea58"

# 应用为：智能问题解决者
app_id = "b9473e78-754b-463a-916b-f0a9097a8e5f"
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

##### 继承AppBuilderEventHandler类，并实现针对各类型event的处理方法

In [2]:
from appbuilder.core.console.appbuilder_client.event_handler import AppBuilderEventHandler
class MyEventHandler(AppBuilderEventHandler):
    def execute_local_command(self, cmd: str):
        import subprocess
        try:
            result = subprocess.check_output(cmd, shell=True).decode("utf-8")
            if result.strip() == "":
                return "命令执行成功，无返回值"
            return result
        except Exception as e:
            return str(e)
    
    def interrupt(self, run_context, run_response):
        thought = run_context.current_thought
        # 绿色打印
        print("\033[1;32m", "-> Agent 中间思考: ", thought, "\033[0m")

        tool_output = []
        for tool_call in run_context.current_tool_calls:
            tool_call_id = tool_call.id
            tool_res = self.execute_local_command(
                **tool_call.function.arguments)
            # 蓝色打印
            print("\033[1;34m", "-> 本地ToolCall结果: \n", tool_res, "\033[0m\n")
            tool_output.append(
                {
                    "tool_call_id": tool_call_id,
                    "output": tool_res
                }
            )
        return tool_output
    
    def success(self, run_context, run_response):
        print("\n\033[1;31m","-> Agent 非流式回答: \n", run_response.answer, "\033[0m")

##### 定义本地的tools工具

通过`subprocess.check_output`方法，可以在终端中执行命令，并返回执行结果

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_local_command",
            "description": "可以在bash环境中，执行输入的指令，注意，一次只能执行一个原子命令。例如：ls",
            "parameters": {
                "type": "object",
                "properties": {
                    "cmd": {
                        "type": "string",
                        "description": "需要执行的指令",
                    },
                },
                "required": ["cmd"],
            },
        },
    }
]

with app_client.run_with_handler(
        conversation_id = conversation_id,
        query = "请问当前文件夹下有哪些文件？如果没有test.txt文件，请新建一个test.txt文件，内容为：Hello World！",
        tools = tools,
        event_handler = MyEventHandler(),
    ) as run:
        run.until_done()

 -> Agent 中间思考:   
 -> 本地ToolCall结果: 
 app_builder_resources
appbuilder_client.ipynb
tmp.log
tool_call.ipynb
tool_choice.ipynb
 


 -> Agent 非流式回答: 
 当前文件夹下的文件有：

- app_builder_resources
- appbuilder_client.ipynb
- tmp.log
- tool_call.ipynb
- tool_choice.ipynb

如果没有 `test.txt` 文件，我已经为您新建了一个 `test.txt` 文件，内容为：Hello World！ 


**output**
```
 -> Agent 中间思考:  首先，我需要使用execute_local_command工具来执行'ls'命令，列出当前文件夹下的所有文件。然后，我需要检查输出中是否存在test.txt文件。如果不存在，我将再次使用execute_local_command工具来执行'echo "Hello World" > test.txt'命令，以创建并写入test.txt文件。 
 -> 本地ToolCall结果: 
 multi_tool_call.ipynb
multi_tool_call.py
multi_tool_call_with_handler.ipynb
multi_tool_call_with_handler.py
sdk_ knowledgebase.ipynb
sdk_trace.ipynb
simple_tool_call.ipynb
simple_tool_call.py
tmp.log
黑神话(悟空).pdf
 

 -> Agent 中间思考:  根据execute_local_command工具的返回结果，当前文件夹下并没有test.txt文件。因此，我需要使用execute_local_command工具来执行'echo "Hello World" > test.txt'命令，以创建并写入test.txt文件。 
 -> 本地ToolCall结果: 
 命令执行成功，无返回值 


 -> Agent 非流式回答: 
 当前文件夹下的文件包括：

- multi_tool_call.ipynb
- multi_tool_call.py
- multi_tool_call_with_handler.ipynb
...
- tmp.log
- 黑神话(悟空).pdf

经过检查，发现当前文件夹下**不存在**test.txt文件。因此，已经为您新建了一个test.txt文件，并写入了内容“Hello World！”。 
```

- 使用AppBuilderEventHandler架构可以简化client的交互方式

# 6、项目总结

本项目通过多个知识点的学习，以及两个使用AppBuilder-SDK的实操，最终完成了一个支持ToolCall AIAgent的构建。

- 理论学习：了解AIAgent的基础知识
- 上手实操：深入了解Agent中的FunctionCall运行流程
- 上手实操：入门百度智能云千帆AppBuilder，在十分钟内打造一个个性化AIAgent
- 上手实操：使用AppBuilder-SDK打造一个端云组件联动的进阶Agent


希望您可以不吝`Star`，给`AppBuilder-SDK`一些鼓励，期待您的`PR`，一起共建AIAgent生态。

Github地址：https://github.com/baidubce/app-builder

<img src="https://chengmo-dev1.bj.bcebos.com/page10.png" alt="drawing" width="1000"/>

最后，您也可以进入`AppBuilder-SDK`的WX交流群，和大家一起交流AppBuilder使用及开发心得。

<img src="https://chengmo-dev1.bj.bcebos.com/wechat_group.png" alt="drawing" width="1000"/>